In [ ]:
%cd ..

# Tester
resultsからテストする  
予測融合はこれでテスト

In [ ]:
from mylib.data import my_dataloader, my_dataset
from mylib.classification import my_cmatrix, my_roc

In [ ]:
import sys
import pandas as pd
import torch

In [ ]:
print(f"Python version: {sys.version}")
print(f"Torch version:  {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"device name:    {torch.cuda.get_device_name()}")

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## I/O

In [ ]:
image_path  = "data/images"
label_path  = "data/labels/label_legacy.xlsx"
results_inpath = "main/outputs/ProbFusion/results"  # resultsのパス
output_path = "main/outputs/ProbFusion"

fname_key = "fname"
label_key = "pap2_groundtruth"  # テストラベル
fold_key  = "slide"

name_classes = ['positive','negative']
num_classes  = len(name_classes)

## Parameters

In [ ]:
image_size = 224
batch_size = 64
val_ratio  = 0.1

## Data

In [ ]:
# データのロード
_images, _labels, _fnames, _folds = my_dataloader.loadData4(
    image_path,
    label_path,
    fname_key = fname_key,
    label_key = label_key,
    fold_key  = fold_key,
    resize    = image_size,
    to_tensor = True
)
print(len(_fnames))


# 不適切なデータを除去
images, labels, fnames, folds = [], [], [], []
for i in range(len(_images)):
    if _labels[i] != -1:
        images.append(_images[i])
        labels.append(_labels[i])
        fnames.append(_fnames[i])
        folds.append(_folds[i])

In [ ]:
# データセットの作成
dataset = my_dataset.MyDataset(images, labels, fnames, name_classes)

# Leave-one-case-outマネージャーを作成
managers = my_dataset.getLocoManager(folds)

## Test

In [ ]:
for manager in managers:
    fold_id, fold_name = manager['fold_id'], manager['fold_name']
    print(f"======== fold-id: {fold_id} / fold-name: {fold_name} ========")

    # resultsを読み込む
    results = pd.read_excel(f"{results_inpath}/{fold_name}_results.xlsx")
    
    # 予測結果の解析
    f, t, p, n = results['fname'], results['true'], results['positive'], results['negative']
    
    # 確信度の分布
    my_roc.plotDistributionHistogram(t, p, save_string=f"{output_path}/prob_dist/{fold_name}_probdist.png")
    
    # ROC
    thresh, fpr, tpr = my_cmatrix.get_youden_index(t, n)
    my_roc.plotROCwithYoudenPoint(t, n, name_classes, fpr, tpr, save_string=f"{output_path}/roc/{fold_name}_roc.png")
    
    # 混同行列
    my_cmatrix.plotConfusionMatrix(t, p, name_classes, t=thresh, save_string=f"{output_path}/cmatrix/{fold_name}_cmatrix.png")
    
    # Youden-Indexのカットオフ値による予測
    preds_cutoff = [0 if thresh <= x else 1 for x in p]
    youden_pred = {"fname": f,"true": t,"pred": preds_cutoff}
    df_youden_pred = pd.DataFrame.from_dict(youden_pred)
    df_youden_pred.to_excel(f"{output_path}/preds/{fold_name}_youden-preds.xlsx", index=False)
    print()